In [1]:
import numpy as np

from dl_ts_models.main import create_classifier
from drowsiness_detection import config
from drowsiness_detection.data import load_experiment_data
from drowsiness_detection.models import ThreeDStandardScaler

In [11]:
# load data
config.set_paths(30, 10)
X_train, X_test, y_train, y_test, cv_splits = load_experiment_data(
    exclude_by="a",
    num_targets=2,
    seed=45,
    test_size=.2,
    split_by_subjects=True,
    use_dummy_data=False,
    nn_experiment=True,
    feature_col_indices=(5, 8, 9, 14, 15, 16, 19),
    model_name="mvts_transformer")

X_train shape: (29012, 300, 7), y_train shape: (29012,)
X_test shape: (7094, 300, 7), y_test shape: (7094,)


In [12]:
num_samples = None
X_train = X_train[:num_samples]
X_test = X_test[:num_samples]
y_test = y_test[:num_samples]
y_train = y_train[:num_samples]



In [13]:
scaler = ThreeDStandardScaler(feature_axis=-1)

X_train_scaled = scaler.fit_transform(X_train, y_train)
X_test_scaled = scaler.transform(X_test)

input_shape = (None, 300, 7)


In [16]:
# base_path = config.SOURCES_ROOT_PATH.parent.joinpath("data/gcloud_dataset/30sec/")
# np.save(base_path.joinpath("x_test.npy"), X_test_scaled)
# np.save(base_path.joinpath("x_train.npy"), X_train_scaled)
# np.save(base_path.joinpath("y_test.npy"), y_test)
# np.save(base_path.joinpath("y_train.npy"), y_train)
X_test[0]

array([[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         4.9020001e+02,  3.0344245e-01,  0.0000000e+00],
       [ 0.0000000e+00,  6.2534122e+00, -1.9878553e+01, ...,
         4.9310001e+02,  3.0124053e-01,  0.0000000e+00],
       [ 0.0000000e+00, -2.5014088e+00, -3.7266901e+00, ...,
         4.7529999e+02,  3.0444688e-01,  0.0000000e+00],
       ...,
       [ 0.0000000e+00, -2.5013887e+01,  7.3306213e+01, ...,
         5.0939999e+02,  2.8338018e-01,  0.0000000e+00],
       [ 0.0000000e+00,  2.8766104e+01, -3.7274466e+00, ...,
         5.0850000e+02,  2.9527017e-01,  0.0000000e+00],
       [ 0.0000000e+00,  2.1262104e+01,  1.8637159e+01, ...,
         5.1679999e+02,  2.8618336e-01,  0.0000000e+00]], dtype=float32)

In [5]:
model = create_classifier(
    classifier_name="resnet",
    input_shape=input_shape[1:],
    nb_classes=1,
    output_directory=str(config.SOURCES_ROOT_PATH.parent.joinpath("data/resnet_experiments")) + "/",
    verbose=True,
)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 300, 7)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 300, 64)      3648        ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 300, 64)     256         ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 300, 64)      0           ['batch_normalization[0][0]']

In [ ]:

metrics = model.fit(x_train=X_train, y_train=y_train, x_val=X_test, y_val=y_test,
                    nb_epochs=2, batch_size=64,
                    class_weight={0: 0.84, 1: 1.14})
print(metrics)

Epoch 1/2
 32/454 [=>............................] - ETA: 4:21 - loss: 0.7022 - accuracy: 0.4233

In [ ]:
y_pred = model.predict(X_test_scaled, y_test)

In [ ]:
np.argmax([[0], [1]], axis=1)

In [ ]:
from sklearn.metrics import RocCurveDisplay

y_hat = model.predict(X_test_scaled)

RocCurveDisplay.from_predictions(y_pred=y_hat, y_true=y_test, pos_label=1)